In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv


load_dotenv()  # take environment variables

from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

client = AzureOpenAI(
    azure_ad_token_provider=token_provider
)

image_url = os.environ["IMAGE_SAMPLE_URL"]
image_format = os.environ["FILE_SUFFIX"]

from urllib.request import urlopen, Request
import base64

request = Request(image_url, headers={"User-Agent": "Mozilla/5.0"})
image_data = base64.b64encode(urlopen(request).read()).decode("utf-8")
data_url = f"data:image/{image_format};base64,{image_data}"

In [ ]:
import requests
import IPython.display as Disp

Disp.Image(requests.get(image_url).content)

In [ ]:
prompt = os.environ["PROMPT"].replace("\\n","\n").replace("\\t","\t")

In [ ]:
response = client.chat.completions.create(
    model=os.environ["OPENAI_MODEL_NAME"],
    messages=[
        {
            "role": "user",
            "content": [ {
                "type": "text",
                "text": prompt
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": data_url
                }
            }]
        }
    ]
)

In [ ]:
print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")
print("Model:", response.model)
print("Usage:")
print("\tPrompt tokens:", response.usage.prompt_tokens)
print("\tCompletion tokens:", response.usage.completion_tokens)
print("\tTotal tokens:", response.usage.total_tokens)